# Zadanie nr 2 - Trie i drzewa suffiksów

In [5]:
from treelib import Tree
from time import time
import matplotlib.pyplot as plt

1. Przyjmij następujący zbiór danych wejściowych:
>* bbbd
>* aabbabd
>* ababcd
>* abcbccd
>* załączony plik.

In [6]:
test_data = ['bbbd', 'aabbabd', 'ababcd', 'abcabccd']

with open('1997_714_head.txt', 'r') as file:
    text = "".join(file)
    test_data.append(text)

2. Upewnij się, że każdy łańcuch na końcu posiada unikalny znak (marker), a jeśli go nie ma, to dodaj ten znak.

Do każdego łańcucha dodaję na końcu znak "$", aby było spełnione powyższe kryterium.

3. Zaimplementuj algorytm konstruujący strukturę trie, która przechowuje wszystkie sufiksy łańcucha danego na wejściu.

In [ ]:
class TrieNode:
    def __init__(self, char):
        self.char = char
        self.children = {}

    def __str__(self):
        return self.char

    def __lt__(self, other):
        return self.char < other.char

In [ ]:
def build_trie(text):
    text += '\$'
    head = TrieNode("")

    for i in range(len(text)):
        current_node = head
        for j, char in enumerate(text[i:]):
            if char in current_node.children:
                current_node = current_node.children[char]
            else:
                for k, char in enumerate(text[i+j:]):
                    new_node = TrieNode(char)
                    current_node.children[char] = new_node
                    current_node = new_node
                break

    return head

In [ ]:
def show_trie(text):
    tree = Tree()
    head = build_trie(text)
    tree.create_node(head, head, parent=None)

    def create_tree(node):
        for child in node.children.values():
            tree.create_node(child, child, parent=node)
            create_tree(child)

    create_tree(head)
    tree.show()


4. Zaimplementuj algorytm konstruujący drzewo sufiksów.
Upewnij się, że powstałe struktury danych są poprawne. Możesz np. sprawdzić, czy struktura zawiera jakiś ciąg znaków i porównać wyniki z algorytmem wyszukiwania wzorców.


In [7]:
class TreeNode:
    def __init__(self, range):
        self.range = range
        self.children = set()

    def __str__(self):
        str = ""
        for i in range(self.range[0], self.range[1]+1):
            str += TreeNode.text[i]
        return str

    def __repr__(self):
        str = ""
        for i in range(self.range[0], self.range[1]+1):
            str += TreeNode.text[i]
        return str

    def __lt__(self, other):
        return str(self) < str(other)

In [ ]:
def build_tree(text):
    text += '$'
    head = TreeNode(range=(0, -1))

    for suffix_index in range(len(text)):
        current_node = head
        current_index = suffix_index
        found = 0
        while current_index < len(text) and not found:
            skip = 0
            for child in current_node.children:
                common = 0

                for k in range(child.range[0], child.range[1]+1):
                    if text[suffix_index+common] == text[k]:
                        common += 1
                    else:
                        break

                if common == child.range[1] - child.range[0] + 1:
                    current_node = child
                    current_index += common
                    skip = 1

                elif common > 0:
                    new_child_1 = TreeNode(
                        range=(child.range[0]+common, child.range[1]))
                    new_child_2 = TreeNode(
                        range=(current_index+common, len(text)-1))

                    new_child_1.children = child.children

                    child.range = (
                        child.range[0], child.range[0]+common-1)
                    child.children = {new_child_1, new_child_2}
                    found = 1
                    break

            if not found and not skip:
                new_child = TreeNode(range=(current_index, len(text)-1))
                current_node.children.add(new_child)
                break

    return head

In [ ]:
def show_trie(text):
    tree = Tree()
    head = build_trie(text)
    tree.create_node(head, head, parent=None)

    def create_tree(node):
        for child in node.children.values():
            tree.create_node(child, child, parent=node)
            create_tree(child)

    create_tree(head)
    tree.show()

5. Porównaj szybkość działania algorytmów konstruujących struktury danych dla danych z p. 1 w następujących wariantach:
    * Trie (1 pkt) - czas budowy drzewa O(n^2), rozmiar drzewa O(n^2), n - długość tekstu, przy założeniu, że rozmiar alfabetu jest stały
    * Drzewo sufiksów bez wykorzystania procedury fast_find oraz elementów "link" (2 pkt) - czas budowy O(n^2), rozmiar drzewa O(n)


6. Dla zinteresowanych: implementacja i testy w opraciu o algorytm McCreighta lub Ukkonena (+2 pkt) (czas budowy i rozmiar drzewa O(n)).

Marcin Hawryluk, 03.04.2021